# Instalasi

In [51]:
# import library
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import pickle as pkl

sns.set()

from sklearn import metrics
from sklearn.metrics import classification_report, mean_absolute_error, mean_squared_error, r2_score
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor

In [52]:
# Load data
df = pd.read_excel('D:\SEMESTER7/harga_tiket.xlsx')
df.head()

<>:2: SyntaxWarning: invalid escape sequence '\S'
<>:2: SyntaxWarning: invalid escape sequence '\S'
C:\Users\dinda\AppData\Local\Temp\ipykernel_6532\170879067.py:2: SyntaxWarning: invalid escape sequence '\S'
  df = pd.read_excel('D:\SEMESTER7/harga_tiket.xlsx')


,extract_timestamp,origin,destination,depart_date,best_price
0,2022-12-29 03:13:11.798742,JKTC,BTJ,2023-01-01,1943630.00
1,2022-12-29 03:13:11.798742,JKTC,BDO,2023-01-01,2542850.00
2,2022-12-29 03:13:11.798742,JKTC,MLG,2023-01-01,1312650.00
3,2022-12-29 03:13:11.798742,JKTC,SRG,2023-01-01,688180.00
4,2022-12-29 03:13:11.798742,JKTC,SUB,2023-01-01,676230.00


In [53]:
#Menghilangkan .0 pada kolom best_price
df['best_price'] = df['best_price'].astype(int)
print(df)

                extract_timestamp origin destination depart_date  best_price
0      2022-12-29 03:13:11.798742   JKTC         BTJ  2023-01-01     1943630
1      2022-12-29 03:13:11.798742   JKTC         BDO  2023-01-01     2542850
2      2022-12-29 03:13:11.798742   JKTC         MLG  2023-01-01     1312650
3      2022-12-29 03:13:11.798742   JKTC         SRG  2023-01-01      688180
4      2022-12-29 03:13:11.798742   JKTC         SUB  2023-01-01      676230
...                           ...    ...         ...         ...         ...
45433  2023-01-01 02:09:16.289170   JKTC         JOG  2024-06-11     1122400
45434  2023-01-01 02:09:16.289170   JKTC         YIA  2024-06-11      367200
45435  2023-01-01 02:09:16.289170   JKTC         DPS  2024-06-22      697900
45436  2023-01-01 02:09:16.289170   JKTC         JOG  2024-06-26     1122400
45437  2023-01-01 02:09:16.289170   JKTC         YIA  2024-06-26      367200

[45438 rows x 5 columns]


In [54]:
# Mengubah format file pada kolom 'depart_date' menjadi datetime
df["depart_date"] = pd.to_datetime(df["depart_date"], format="%Y-%m-%d")

# Extract useful features from 'depart_date'
df["year"] = df["depart_date"].dt.year              # Year
df["month"] = df["depart_date"].dt.month            # Month (1-12)
df["day"] = df["depart_date"].dt.day                # Day of the month

# Preprocessing

## Cek Missing Values

In [55]:
# Menampilkan jumlah nilai yang hilang untuk setiap kolom
missing_values = df.isnull().sum()
print(missing_values)

extract_timestamp    0
origin               0
destination          0
depart_date          0
best_price           0
year                 0
month                0
day                  0
dtype: int64


## One Hot Encoding

In [56]:
# Encode 'destination' using OneHotEncoder
encoder = OneHotEncoder(sparse_output=False)
destination_encoded = encoder.fit_transform(df[["destination"]])

destination_encoded_df = pd.DataFrame(destination_encoded, columns=encoder.get_feature_names_out(["destination"]))

In [57]:
destination_encoded_df = destination_encoded_df.astype("int32")
print(destination_encoded_df)

       destination_BDJ  destination_BDO  destination_BIK  destination_BKS  \
0                    0                0                0                0   
1                    0                1                0                0   
2                    0                0                0                0   
3                    0                0                0                0   
4                    0                0                0                0   
...                ...              ...              ...              ...   
45433                0                0                0                0   
45434                0                0                0                0   
45435                0                0                0                0   
45436                0                0                0                0   
45437                0                0                0                0   

       destination_BPN  destination_BTH  destination_BTJ  destination_DJB  

In [58]:
# Save model
import joblib
joblib.dump(destination_encoded_df, 'encode.pkl')
print("Model saved!")

Model saved!


In [59]:
# Combine encoded 'destination' with the rest of the dataset
df_preprocessed = pd.concat([df.drop(columns=["origin","destination", "extract_timestamp", "depart_date"]), destination_encoded_df], axis=1)
print(df_preprocessed)

       best_price  year  month  day  destination_BDJ  destination_BDO  \
0         1943630  2023      1    1                0                0   
1         2542850  2023      1    1                0                1   
2         1312650  2023      1    1                0                0   
3          688180  2023      1    1                0                0   
4          676230  2023      1    1                0                0   
...           ...   ...    ...  ...              ...              ...   
45433     1122400  2024      6   11                0                0   
45434      367200  2024      6   11                0                0   
45435      697900  2024      6   22                0                0   
45436     1122400  2024      6   26                0                0   
45437      367200  2024      6   26                0                0   

       destination_BIK  destination_BKS  destination_BPN  destination_BTH  \
0                    0                0       

In [81]:
import joblib
joblib.dump(df_preprocessed, 'encoder.joblib')

['encoder.joblib']

# Pembagian Data

## Pemisahan Kolom Independen dan Dependen

Pemisahan kolom independen dan dependen ini dilakukan agar nantinya data siap diolah pada proses selanjutnya yaitu proses pembagian data latih dan tes. Pemisahan ini akan menghasilkan dua variabel data baru, yaitu X yang berisi data independen, dan y yang berisi data dependen

In [60]:
# Memisahkan fitur X dan target y
X = df_preprocessed.drop(columns=["best_price"])
y = df_preprocessed['best_price']

In [61]:
# Menampilkan 5 baris data X (dependen)
X.head()

,year,month,day,destination_BDJ,destination_BDO,destination_BIK,destination_BKS,destination_BPN,destination_BTH,destination_BTJ,...,destination_PLM,destination_PNK,destination_SOC,destination_SRG,destination_SUB,destination_TKG,destination_TNJ,destination_TRK,destination_UPG,destination_YIA
0,2023,1,1,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
1,2023,1,1,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2023,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,2023,1,1,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
4,2023,1,1,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0


In [62]:
# Menampilkan 5 baris data y (independen)
y.head()

0    1943630
1    2542850
2    1312650
3     688180
4     676230
Name: best_price, dtype: int32

## Split Data

Pada proses ini data akan dibagi menjadi dua, yaitu data latih dan data tes. Data latih digunakan untuk melatih model, sementara data tes digunakan untuk menguji kinerja model yang telah dilatih. Pembagian data ini penting untuk menghindari overfitting dan untuk menguji model terhadap data baru yang belum pernah dilihat sebelumnya. Data yang telah melalui pra-pemrosesan data akan dibagi menjadi data latih dan data tes dengan total 4 variabel, yaitu data independen latih (X_train), data independen tes (X_test), data dependen latih (y_train), dan data dependen tes (y_test), dengan test_size yang disesuaikan dari proses sebelumnya.

In [63]:
#Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Pemodelan

## Random Forest Regression

Random Forest Regression adalah algoritma machine learning berbasis ensemble yang digunakan untuk memprediksi nilai kontinu (regresi). Algoritma ini bekerja dengan membangun banyak decision trees selama pelatihan dan menggabungkan hasil prediksi dari semua tree untuk menghasilkan prediksi akhir, biasanya dengan mengambil rata-rata.

In [64]:
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

# Buat objek Random Forest Regressor
model_rf = RandomForestRegressor(n_estimators=100, random_state=42)

# Buat objek KFold cross-validation
kf = KFold(n_splits=5, shuffle=True, random_state=42)

# Inisialiasi metrik evaluasi
training_accuracy = []
testing_accuracy = []
mse_list = []
rmse_list = []
mae_list = []
r2_list = []

# Loop setiap fold
for fold, (train_index, test_index) in enumerate(kf.split(X)):
  # Gunakan .iloc untuk memilih baris berdasarkan indeks integer
  X_train, X_test = X.iloc[train_index], X.iloc[test_index]
  y_train, y_test = y.iloc[train_index], y.iloc[test_index]

  # Latih model
  model_rf.fit(X_train, y_train)

  # Menghitung akurasi untuk Data Pelatihan dan Pengujian
  train_predictions = model_rf.predict(X_train)
  test_predictions = model_rf.predict(X_test)

  # Hitung ulang prediksi dari model Random Forest
  predictions_rf = model_rf.predict(X_test) 

  # Menghitung metrik evaluasi
  mse = mean_squared_error(y_test, predictions_rf)
  rmse = np.sqrt(mse)
  mae = mean_absolute_error(y_test, predictions_rf)
  r2 = r2_score(y_test, predictions_rf)

   # Menghitung akurasi
  training_accuracy.append(model_rf.score(X_train, y_train))
  testing_accuracy.append(model_rf.score(X_test, y_test))

  # Menambahkan list yang ada
  mse_list.append(mse)
  rmse_list.append(rmse)
  mae_list.append(mae)
  r2_list.append(r2)

  # Membulatkan dan mengubah tipe data
  df_pred_rf = pd.DataFrame({'predictions_rf' : predictions_rf})
  df_pred_rf["predictions_rf"] = df_pred_rf["predictions_rf"].round().astype(int)

  print(f"Fold {fold+1}:")
  print(f"  Training Accuracy: {training_accuracy[-1]:.2f}")
  print(f"  Testing Accuracy: {testing_accuracy[-1]:.2f}")
  print(f"  MSE: {mse:.2f}")
  print(f"  RMSE: {rmse:.2f}")
  print(f"  MAE: {mae:.2f}")
  print(f"  R2 Score: {r2:.2f}")
  print(df_pred_rf.head())
  print()

Fold 1:
  Training Accuracy: 0.97
  Testing Accuracy: 0.96
  MSE: 46588568857.61
  RMSE: 215843.85
  MAE: 115896.47
  R2 Score: 0.96
   predictions_rf
0         2576496
1          782936
2          690030
3         1084674
4         1390813

Fold 2:
  Training Accuracy: 0.97
  Testing Accuracy: 0.96
  MSE: 44247713350.02
  RMSE: 210351.40
  MAE: 112471.56
  R2 Score: 0.96
   predictions_rf
0         1991947
1          527054
2         1164409
3         2786437
4          714861

Fold 3:
  Training Accuracy: 0.97
  Testing Accuracy: 0.96
  MSE: 45945617263.13
  RMSE: 214349.29
  MAE: 113975.34
  R2 Score: 0.96
   predictions_rf
0          717548
1         1024853
2          630475
3          709493
4          987282

Fold 4:
  Training Accuracy: 0.97
  Testing Accuracy: 0.96
  MSE: 48098244452.54
  RMSE: 219313.12
  MAE: 116737.52
  R2 Score: 0.96
   predictions_rf
0         1313858
1          760770
2         1632387
3         1216917
4          948520

Fold 5:
  Training Accuracy: 0.9

In [65]:
# Hitung dan cetak rata-rata standar deviasi untuk setiap metrik
print("Overall Metrics:")
print(f"  Training Accuracy: {np.mean(training_accuracy):.2f} +/- {np.std(training_accuracy):.2f}")
print(f"  Testing Accuracy: {np.mean(testing_accuracy):.2f} +/- {np.std(testing_accuracy):.2f}")
print(f"  MSE: {np.mean(mse_list):.2f} +/- {np.std(mse_list):.2f}")
print(f"  RMSE: {np.mean(rmse_list):.2f} +/- {np.std(rmse_list):.2f}")
print(f"  MAE: {np.mean(mae_list):.2f} +/- {np.std(mae_list):.2f}")
print(f"  R2 Score: {np.mean(r2_list):.2f} +/- {np.std(r2_list):.2f}")

Overall Metrics:
  Training Accuracy: 0.97 +/- 0.00
  Testing Accuracy: 0.96 +/- 0.00
  MSE: 46516763013.64 +/- 1370385666.68
  RMSE: 215653.87 +/- 3189.04
  MAE: 115285.17 +/- 1808.74
  R2 Score: 0.96 +/- 0.00


In [66]:
# Prediksi variabel dependen dari data test
prediksi_rf = model_rf.predict(X_test)
# Menampilkan hasil prediksi
prediksi_rf

array([ 843209.45166667,  899301.40366667,  934987.62668254, ...,
       1127152.        ,  431568.16219059,  733343.64438894])

In [67]:
# Mengatur format tampilan
pd.set_option('display.float_format', '{:.2f}'.format)

In [68]:
# Membulatkan dan mengubah tipe data
df_pred_rf = pd.DataFrame({'prediksi_rf' : prediksi_rf})
df_pred_rf["prediksi_rf"] = df_pred_rf["prediksi_rf"].round().astype(int)

# Menampilkan hasil
print(df_pred_rf.head())

   prediksi_rf
0       843209
1       899301
2       934988
3      1055317
4      1279743


In [69]:
test_df = pd.DataFrame({
    "Predicted Price" : model_rf.predict(X_test),
    "Actual Price" : y_test,
}).reset_index(drop = True)

test_df

,Predicted Price,Actual Price
0,843209.45,1312650
1,899301.40,804880
2,934987.63,984850
3,1055317.09,936450
4,1279742.73,961450
...,...,...
9082,1002461.03,1023100
9083,1127680.00,1122400
9084,1127152.00,1122400
9085,431568.16,367200


# Input Prediksi

In [70]:
# Fungsi untuk memproses input tujuan
def preprocess_destination(destination):
    destination_encoded = encoder.transform([[destination]])
    return pd.DataFrame(destination_encoded, columns=encoder.get_feature_names_out(["destination"]))

# Fungsi untuk memprediksi harga berdasarkan tujuan
def predict_price(destination, year, month, day):
    destination_features = preprocess_destination(destination)
    input_data = pd.DataFrame({
        "year": [year],
        "month": [month],
        "day": [day]
    })
    processed_input = pd.concat([input_data, destination_features], axis=1)
    prediction = model_rf.predict(processed_input)
    return prediction[0]

# Input dan Prediksi
def input_destination():
    destination = input("Masukkan tujuan: ")

    # Validasi input tahun
    while True:
        year_input = input("Masukkan tahun (misal, 2023): ")
        if year_input.isdigit():
            year = int(year_input)
            break
        else:
            print("Input tidak valid! Harap masukkan tahun yang valid.")

    # Validasi input bulan
    while True:
        month_input = input("Masukkan bulan (1-12): ")
        if month_input.isdigit() and 1 <= int(month_input) <= 12:
            month = int(month_input)
            break
        else:
            print("Input tidak valid! Harap masukkan bulan yang valid (1-12).")

    # Validasi input tanggal
    while True:
        day_input = input("Masukkan tanggal: ")
        if day_input.isdigit() and 1 <= int(day_input) <= 31:  # Anda bisa menambahkan validasi tanggal berdasarkan bulan
            day = int(day_input)
            break
        else:
            print("Input tidak valid! Harap masukkan tanggal yang valid (1-31).")
    
    # Memanggil fungsi prediksi
    price_prediction = predict_price(destination, year, month, day)
    return price_prediction

# Penggunaan contoh
example_destination = input("Masukkan tujuan untuk prediksi: ")
predicted_price = input_destination()
print(f"Prediksi Harga untuk tujuan {example_destination}: {predicted_price}")

C:\Users\dinda\anaconda3\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


Prediksi Harga untuk tujuan YIA: 547594.1533333333


In [ ]:
# Input manual (berdasarkan input prediksi sebelumnya)
destination = "YIA"
year = 2023
month = 1
day = 1

# Temukan nilai aktual di dataset
actual_price = df[
    (df["destination"] == destination) &
    (df["year"] == year) &
    (df["month"] == month) &
    (df["day"] == day)
]["best_price"]

if not actual_price.empty:
    print(f"Actual Price for destination {destination} on {year}-{month}-{day}: {actual_price.values[0]}")
    predicted_price = predict_price(destination, year, month, day)  # Misalnya day_of_week=4
    print(f"Predicted Price: {predicted_price}")
    print(f"Error: {abs(actual_price.values[0] - predicted_price)}")
else:
    print("No actual data found for the given input.")

Actual Price for destination YIA on 2023-1-1: 569430
Predicted Price: 547594.1533333333
Error: 21835.84666666668


C:\Users\dinda\anaconda3\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


# Simpan Model

In [ ]:
import joblib
joblib.dump(model_rf, 'rf.joblib')